In [2]:
pip install pandas pdfplumber sentence-transformers faiss-cpu transformers torch scikit-learn numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import pdfplumber
import os
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics import precision_score, recall_score
import torch

# Пути к данным
DATASET_PATH = r"C:\Users\funny\OneDrive\Desktop\4Course_Project_HSE\Diploma\For_Data_RAG\RAG_data.csv"
CORPUS_PATH = r"C:\Users\funny\OneDrive\Desktop\4Course_Project_HSE\Diploma\For_Data_RAG\corpus_files"

# Создаем выходную директорию для результатов, если нужно
OUTPUT_DIR = "rag_results"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [4]:
# Чтение CSV
df = pd.read_csv(DATASET_PATH)

# Проверка столбцов
expected_columns = ["Вопрос","Ответ","ИсточникФайл","НачалоСимвола","КонецСимвола","ИзвлеченныйСниппет"]
for col in expected_columns:
    if col not in df.columns:
        raise ValueError(f"Столбец {col} отсутствует в датасете")

# Очистка данных
df = df.dropna()  # Удаляем строки с пропусками
df['question'] = df['Вопрос'].str.strip()
df['answer'] = df['Ответ'].str.strip()
df['snippet'] = df['ИзвлеченныйСниппет'].str.strip()

# Статистика датасета
print(f"Размер датасета: {len(df)} пар вопрос-ответ")
print(f"Уникальных вопросов: {df['question'].nunique()}")
print(f"Средняя длина сниппета: {df['snippet'].str.len().mean():.2f} символов")

# Сохранение очищенного датасета (опционально)
df.to_csv(os.path.join(OUTPUT_DIR, "cleaned_dataset.csv"), index=False)

Размер датасета: 402 пар вопрос-ответ
Уникальных вопросов: 402
Средняя длина сниппета: 454.63 символов


In [5]:
def extract_text_from_pdf(pdf_path):
    """Извлечение текста из PDF-файла."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            text = ""
            for page in pdf.pages:
                text += page.extract_text() or ""
        return text.strip()
    except Exception as e:
        print(f"Ошибка при обработке {pdf_path}: {e}")
        return ""

# Словарь для хранения текста PDF
corpus_texts = {}

# Обработка всех PDF в папке corpus_files
for pdf_file in os.listdir(CORPUS_PATH):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(CORPUS_PATH, pdf_file)
        corpus_texts[pdf_file] = extract_text_from_pdf(pdf_path)

# Проверка извлеченного текста
print(f"Обработано PDF-файлов: {len(corpus_texts)}")

# Сохранение извлеченного текста (опционально)
with open(os.path.join(OUTPUT_DIR, "corpus_texts.txt"), "w", encoding="utf-8") as f:
    for pdf_name, text in corpus_texts.items():
        f.write(f"--- {pdf_name} ---\n{text[:500]}...\n\n")

Обработано PDF-файлов: 8


In [6]:
def split_text_into_chunks(text, chunk_size=500):
    """Разбиение текста на блоки заданного размера."""
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0
    
    for word in words:
        current_length += len(word) + 1
        current_chunk.append(word)
        if current_length >= chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_length = 0
    
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    
    return chunks

# Разбиваем каждый PDF на блоки
corpus_chunks = {}
for pdf_name, text in corpus_texts.items():
    corpus_chunks[pdf_name] = split_text_into_chunks(text, chunk_size=500)

# Подготовка списка всех блоков и их метаданных
all_chunks = []
chunk_metadata = []
for pdf_name, chunks in corpus_chunks.items():
    for i, chunk in enumerate(chunks):
        all_chunks.append(chunk)
        chunk_metadata.append({"pdf_name": pdf_name, "chunk_idx": i})

print(f"Всего блоков текста: {len(all_chunks)}")

Всего блоков текста: 13070


In [7]:
# Загрузка модели для эмбеддингов
embedder = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Создание эмбеддингов для сниппетов из датасета
snippet_embeddings = embedder.encode(df['snippet'].tolist(), show_progress_bar=True)
print(f"Размер эмбеддингов сниппетов: {snippet_embeddings.shape}")

# Создание эмбеддингов для блоков корпуса
chunk_embeddings = embedder.encode(all_chunks, show_progress_bar=True)
print(f"Размер эмбеддингов блоков корпуса: {chunk_embeddings.shape}")

# Сохранение эмбеддингов в FAISS индекс
dimension = snippet_embeddings.shape[1]
snippet_index = faiss.IndexFlatL2(dimension)
snippet_index.add(snippet_embeddings)

chunk_index = faiss.IndexFlatL2(dimension)
chunk_index.add(chunk_embeddings)

# Сохранение индексов (опционально)
faiss.write_index(snippet_index, os.path.join(OUTPUT_DIR, "snippet_index.faiss"))
faiss.write_index(chunk_index, os.path.join(OUTPUT_DIR, "chunk_index.faiss"))

Batches: 100%|██████████| 13/13 [00:06<00:00,  2.14it/s]


Размер эмбеддингов сниппетов: (402, 384)


Batches: 100%|██████████| 409/409 [04:36<00:00,  1.48it/s]


Размер эмбеддингов блоков корпуса: (13070, 384)


In [8]:
def retrieve_snippets(query, index, texts, k=5):
    """Поиск топ-k релевантных текстов для запроса."""
    query_embedding = embedder.encode([query])
    distances, indices = index.search(query_embedding, k)
    retrieved = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return retrieved

# Тестирование ретривера на случайном вопросе
sample_question = df['question'].iloc[0]
print(f"Пример вопроса: {sample_question}")
retrieved_snippets = retrieve_snippets(sample_question, snippet_index, df['snippet'].tolist(), k=5)
for i, (snippet, distance) in enumerate(retrieved_snippets, 1):
    print(f"Сниппет {i} (дистанция: {distance:.4f}): {snippet[:100]}...")

# Оценка ретривера
def evaluate_retriever(df, index, texts, k=5):
    """Оценка ретривера по Precision@k и Recall@k."""
    precisions = []
    recalls = []
    
    for idx, row in df.iterrows():
        query = row['question']
        true_snippet = row['snippet']
        retrieved = retrieve_snippets(query, index, texts, k)
        retrieved_texts = [r[0] for r in retrieved]
        
        # Метки: 1 если сниппет релевантный, 0 если нет
        labels = [1 if true_snippet == r else 0 for r in retrieved_texts]
        if sum(labels) > 0:  # Если есть релевантные
            precisions.append(sum(labels) / k)
            recalls.append(1.0)  # Recall@1, если релевантный найден
        else:
            precisions.append(0.0)
            recalls.append(0.0)
    
    return np.mean(precisions), np.mean(recalls)

# Оценка на сниппетах
snippet_precision, snippet_recall = evaluate_retriever(df, snippet_index, df['snippet'].tolist(), k=5)
print(f"Snippet Retriever - Precision@5: {snippet_precision:.4f}, Recall@5: {snippet_recall:.4f}")

Пример вопроса: Что делать, если сосед построил дом слишком близко к моему участку, нарушая нормы?
Сниппет 1 (дистанция: 12.0288): 6. Возмещение за жилое помещение, сроки и другие условия изъятия определяются соглашением с собствен...
Сниппет 2 (дистанция: 12.1584): 3. Собственник жилого помещения несет бремя содержания данного помещения и, если данное помещение яв...
Сниппет 3 (дистанция: 12.9640): 7. Размер платы за содержание жилого помещения в многоквартирном доме, в котором не созданы товарище...
Сниппет 4 (дистанция: 12.9640): 7. Размер платы за содержание жилого помещения в многоквартирном доме, в котором не созданы товарище...
Сниппет 5 (дистанция: 13.0699): 1. Собственники помещений в многоквартирном доме обязаны уплачивать ежемесячные взносы на капитальны...
Snippet Retriever - Precision@5: 0.1517, Recall@5: 0.6393


In [9]:
pip install tiktoken


Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install blobfile

Note: you may need to restart the kernel to use updated packages.


In [11]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Загрузка модели для генерации
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
generator = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

def generate_answer(question, snippets, max_length=200):
    """Генерация ответа на основе вопроса и контекста."""
    context = " ".join([s for s, _ in snippets])
    input_text = f"Вопрос: {question} Контекст: {context}"
    
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = generator.generate(
        **inputs,
        max_length=max_length,
        num_beams=5,
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Тестирование генератора
sample_question = df['question'].iloc[0]
retrieved_snippets = retrieve_snippets(sample_question, snippet_index, df['snippet'].tolist(), k=5)
generated_answer = generate_answer(sample_question, retrieved_snippets)
print(f"Сгенерированный ответ: {generated_answer}")
print(f"Референсный ответ: {df['answer'].iloc[0]}")

c:\Users\funny\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
c:\Users\funny\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\funny\.cache\huggingface\hub\models--google--mt5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the 

Сгенерированный ответ: <extra_id_0>.;
Референсный ответ: В соответствии со статьей 54 Градостроительного кодекса Российской Федерации, в случае выявления нарушения строительных норм, следует обратиться в органы государственного строительного надзора.


In [12]:
pip install sentencepiece


Note: you may need to restart the kernel to use updated packages.


In [13]:
import sentencepiece
print(sentencepiece.__version__)

0.2.0


In [26]:
def retrieve_snippets(query, index, texts, k=5):
    """Поиск топ-k релевантных текстов для запроса."""
    try:
        query_embedding = embedder.encode([query])
        distances, indices = index.search(query_embedding, k)
        # Формируем список кортежей (текст, расстояние)
        retrieved = []
        for j, i in enumerate(indices[0]):
            if i < len(texts):
                retrieved.append((texts[i], float(distances[0][j])))
            else:
                print(f"Предупреждение: индекс {i} вне диапазона texts")
        return retrieved
    except Exception as e:
        print(f"Ошибка в retrieve_snippets: {e}")
        return []

# Тестирование ретривера
try:
    sample_question = df['question'].iloc[0]
    retrieved_snippets = retrieve_snippets(sample_question, snippet_index, df['snippet'].tolist(), k=5)
    print(f"Пример вопроса: {sample_question}")
    print("Результат retrieve_snippets:")
    for i, item in enumerate(retrieved_snippets, 1):
        print(f"Сниппет {i}: {item}")
except Exception as e:
    print(f"Ошибка при тестировании ретривера: {e}")

# Оценка ретривера
def evaluate_retriever(df, index, texts, k=5):
    """Оценка ретривера по Precision@k и Recall@k."""
    precisions = []
    recalls = []
    
    for idx, row in df.iterrows():
        query = row['question']
        true_snippet = row['snippet']
        try:
            retrieved = retrieve_snippets(query, index, texts, k)
            retrieved_texts = [r[0] for r in retrieved if isinstance(r, tuple) and len(r) == 2]
            
            labels = [1 if true_snippet == r else 0 for r in retrieved_texts]
            if sum(labels) > 0:
                precisions.append(sum(labels) / k)
                recalls.append(1.0)
            else:
                precisions.append(0.0)
                recalls.append(0.0)
        except Exception as e:
            print(f"Ошибка при оценке ретривера для вопроса {idx}: {e}")
            continue
    
    precision_mean = np.mean(precisions) if precisions else 0.0
    recall_mean = np.mean(recalls) if recalls else 0.0
    return precision_mean, recall_mean

# Оценка
try:
    snippet_precision, snippet_recall = evaluate_retriever(df, snippet_index, df['snippet'].tolist(), k=5)
    print(f"Snippet Retriever - Precision@5: {snippet_precision:.4f}, Recall@5: {snippet_recall:.4f}")
except Exception as e:
    print(f"Ошибка при оценке ретривера: {e}")

Пример вопроса: Что делать, если сосед построил дом слишком близко к моему участку, нарушая нормы?
Результат retrieve_snippets:
Сниппет 1: ('6. Возмещение за жилое помещение, сроки и другие условия изъятия определяются соглашением с собственником жилого помещения. Принудительное изъятие жилого помещения на основании решения суда возможно только при условии предварительного и равноценного возмещения.; 8. По соглашению с собственником жилого помещения ему может быть предоставлено взамен изымаемого жилого помещения другое жилое помещение с зачетом его стоимости при определении размера возмещения за изымаемое жилое помещение.', 12.02879524230957)
Сниппет 2: ('3. Собственник жилого помещения несет бремя содержания данного помещения и, если данное помещение является квартирой, общего имущества собственников помещений в соответствующем многоквартирном доме, а собственник комнаты в коммунальной квартире несет также бремя содержания общего имущества собственников комнат в такой квартире, если и

In [27]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Проверка sentencepiece
try:
    import sentencepiece
    print(f"SentencePiece установлен, версия: {sentencepiece.__version__}")
except ImportError:
    print("Ошибка: SentencePiece не установлен. Установите его с помощью 'pip install sentencepiece'")
    exit(1)

# Загрузка модели
try:
    tokenizer = T5Tokenizer.from_pretrained("sberbank-ai/ruT5-base")
    generator = T5ForConditionalGeneration.from_pretrained("sberbank-ai/ruT5-base")
    print("Модель ruT5-base успешно загружена")
except Exception as e:
    print(f"Ошибка при загрузке модели: {e}")
    exit(1)

def generate_answer(question, snippets, max_length=300):
    """Генерация ответа на основе вопроса и контекста."""
    try:
        # Проверка snippets
        print(f"Snippets в generate_answer: {[(type(item), item) for item in snippets]}")
        selected_snippets = []
        for item in snippets[:3]:
            if isinstance(item, tuple) and len(item) == 2 and isinstance(item[0], str):
                selected_snippets.append(item[0][:500])
            else:
                print(f"Пропущен некорректный сниппет: {item}")
        
        context = " ".join(selected_snippets) if selected_snippets else "Контекст отсутствует"
        
        input_text = (
            f"Задача: ответить на юридический вопрос, используя предоставленный контекст. "
            f"Вопрос: {question.strip()} "
            f"Контекст: {context.strip()}"
        )
        
        print(f"Входной текст для модели:\n{input_text[:500]}...")
        
        inputs = tokenizer(
            input_text,
            max_length=512,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        
        with torch.no_grad():
            outputs = generator.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=max_length,
                min_length=20,
                num_beams=5,
                no_repeat_ngram_size=3,
                length_penalty=1.5,
                early_stopping=True
            )
        
        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
        return answer.strip() if answer.strip() else "Ответ не сгенерирован"
    
    except Exception as e:
        print(f"Ошибка при генерации ответа: {e}")
        return "Ошибка генерации"

# Тестирование
try:
    sample_question = df['question'].iloc[0]
    retrieved_snippets = retrieve_snippets(sample_question, snippet_index, df['snippet'].tolist(), k=5)
    print(f"Найденные сниппеты:")
    for i, (snippet, dist) in enumerate(retrieved_snippets, 1):
        print(f"Сниппет {i} (дистанция: {dist:.4f}): {snippet[:100]}...")
    generated_answer = generate_answer(sample_question, retrieved_snippets)
    print(f"Пример вопроса: {sample_question}")
    print(f"Сгенерированный ответ: {generated_answer}")
    print(f"Референсный ответ: {df['answer'].iloc[0]}")
except Exception as e:
    print(f"Ошибка при тестировании генератора: {e}")

SentencePiece установлен, версия: 0.2.0
Модель ruT5-base успешно загружена
Найденные сниппеты:
Сниппет 1 (дистанция: 12.0288): 6. Возмещение за жилое помещение, сроки и другие условия изъятия определяются соглашением с собствен...
Сниппет 2 (дистанция: 12.1584): 3. Собственник жилого помещения несет бремя содержания данного помещения и, если данное помещение яв...
Сниппет 3 (дистанция: 12.9640): 7. Размер платы за содержание жилого помещения в многоквартирном доме, в котором не созданы товарище...
Сниппет 4 (дистанция: 12.9640): 7. Размер платы за содержание жилого помещения в многоквартирном доме, в котором не созданы товарище...
Сниппет 5 (дистанция: 13.0699): 1. Собственники помещений в многоквартирном доме обязаны уплачивать ежемесячные взносы на капитальны...
Snippets в generate_answer: [(<class 'tuple'>, ('6. Возмещение за жилое помещение, сроки и другие условия изъятия определяются соглашением с собственником жилого помещения. Принудительное изъятие жилого помещения на основании

In [28]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
import numpy as np

def evaluate_generator(df, retriever_index, texts, k=5):
    """Оценка генератора по BLEU и ROUGE."""
    bleu_scores = []
    rouge = Rouge()
    rouge_scores = []
    
    for idx, row in df.iloc[:50].iterrows():
        question = row['question']
        reference = row['answer']
        try:
            snippets = retrieve_snippets(question, retriever_index, texts, k)
            
            # Отладка
            print(f"Вопрос {idx}, сниппеты:")
            for i, item in enumerate(snippets, 1):
                print(f"Сниппет {i}: {item}")
            
            generated = generate_answer(question, snippets)
            
            if generated in ["Ошибка генерации", "Ответ не сгенерирован", "Контекст отсутствует"] or not generated.strip():
                print(f"Пропущен вопрос {idx}: ошибка генерации")
                continue
            
            # BLEU
            reference_tokens = reference.split()
            generated_tokens = generated.split()
            bleu = sentence_bleu([reference_tokens], generated_tokens, weights=(0.5, 0.5))
            bleu_scores.append(bleu)
            
            # ROUGE
            scores = rouge.get_scores(generated, reference)
            rouge_scores.append(scores['rouge-l']['f'])
            
            # Сохранение результатов
            with open(os.path.join(OUTPUT_DIR, "generated_answers.txt"), "a", encoding="utf-8") as f:
                f.write(f"Вопрос {idx}:\n")
                f.write(f"Текст: {question}\n")
                f.write("Сниппеты:\n")
                for i, item in enumerate(snippets, 1):
                    if isinstance(item, tuple) and len(item) == 2:
                        f.write(f"{i}. {item[0][:50]}... (дистанция: {item[1]:.4f})\n")
                f.write(f"Сгенерированный: {generated}\n")
                f.write(f"Референсный: {reference}\n")
                f.write(f"BLEU: {bleu:.4f}, ROUGE-L: {scores['rouge-l']['f']:.4f}\n\n")
        
        except Exception as e:
            print(f"Ошибка при обработке вопроса {idx}: {e}")
            continue
    
    bleu_mean = np.mean(bleu_scores) if bleu_scores else 0.0
    rouge_mean = np.mean(rouge_scores) if rouge_scores else 0.0
    print(f"Обработано вопросов: {len(bleu_scores)} из 50")
    return bleu_mean, rouge_mean

# Оценка
try:
    bleu_score, rouge_score = evaluate_generator(df, snippet_index, df['snippet'].tolist(), k=5)
    print(f"Generator - BLEU: {bleu_score:.4f}, ROUGE-L: {rouge_score:.4f}")
except Exception as e:
    print(f"Ошибка при оценке генератора: {e}")

Вопрос 0, сниппеты:
Сниппет 1: ('6. Возмещение за жилое помещение, сроки и другие условия изъятия определяются соглашением с собственником жилого помещения. Принудительное изъятие жилого помещения на основании решения суда возможно только при условии предварительного и равноценного возмещения.; 8. По соглашению с собственником жилого помещения ему может быть предоставлено взамен изымаемого жилого помещения другое жилое помещение с зачетом его стоимости при определении размера возмещения за изымаемое жилое помещение.', 12.02879524230957)
Сниппет 2: ('3. Собственник жилого помещения несет бремя содержания данного помещения и, если данное помещение является квартирой, общего имущества собственников помещений в соответствующем многоквартирном доме, а собственник комнаты в коммунальной квартире несет также бремя содержания общего имущества собственников комнат в такой квартире, если иное не предусмотрено федеральным законом или договором.; 4. Собственник жилого помещения обязан поддерживать

In [29]:
import matplotlib.pyplot as plt

# График метрик ретривера
metrics = {
    "Precision@5": snippet_precision,
    "Recall@5": snippet_recall,
}
plt.bar(metrics.keys(), metrics.values())
plt.title("Retriever Metrics")
plt.savefig(os.path.join(OUTPUT_DIR, "retriever_metrics.png"))
plt.close()

# График метрик генератора
metrics = {
    "BLEU": bleu_score,
    "ROUGE-L": rouge_score,
}
plt.bar(metrics.keys(), metrics.values())
plt.title("Generator Metrics")
plt.savefig(os.path.join(OUTPUT_DIR, "generator_metrics.png"))
plt.close()

In [30]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os

def visualize_results(df, retriever_index, texts, output_dir, k=5, bleu_scores=None, rouge_scores=None):
    """Визуализация результатов RAG-системы."""
    try:
        # 1. Подготовка данных для визуализации
        questions = df['question'].iloc[:50].tolist()
        retriever_distances = []
        retriever_precisions = []
        
        # Собираем дистанции и метрики ретривера
        for idx, question in enumerate(questions):
            try:
                snippets = retrieve_snippets(question, retriever_index, texts, k)
                distances = [dist for _, dist in snippets if isinstance(snippets, list) and len(snippets) > 0]
                retriever_distances.append(distances)
                
                # Precision@k (упрощённая версия, если true_snippet недоступен)
                true_snippet = df['snippet'].iloc[idx]
                retrieved_texts = [s for s, _ in snippets if isinstance(s, str)]
                precision = 1.0 if true_snippet in retrieved_texts else 0.0
                retriever_precisions.append(precision)
            except Exception as e:
                print(f"Ошибка при обработке вопроса {idx} для визуализации: {e}")
                retriever_distances.append([])
                retriever_precisions.append(0.0)
        
        # 2. Визуализация дистанций ретривера
        plt.figure(figsize=(10, 6))
        sns.boxplot(data=retriever_distances)
        plt.title("Распределение дистанций ретривера для топ-5 сниппетов")
        plt.xlabel("Ранг сниппета")
        plt.ylabel("Косинусная дистанция")
        plt.savefig(os.path.join(output_dir, "retriever_distances.png"))
        plt.close()
        
        # 3. Визуализация Precision@5
        plt.figure(figsize=(8, 5))
        sns.histplot(retriever_precisions, bins=10)
        plt.title("Распределение Precision@5 ретривера")
        plt.xlabel("Precision@5")
        plt.ylabel("Частота")
        plt.savefig(os.path.join(output_dir, "retriever_precision.png"))
        plt.close()
        
        # 4. Визуализация метрик генератора (если доступны)
        if bleu_scores and rouge_scores:
            plt.figure(figsize=(8, 5))
            plt.scatter(bleu_scores, rouge_scores, alpha=0.5)
            plt.title("BLEU vs ROUGE-L для генератора")
            plt.xlabel("BLEU")
            plt.ylabel("ROUGE-L")
            plt.savefig(os.path.join(output_dir, "generator_metrics.png"))
            plt.close()
        else:
            print("Метрики генератора недоступны, пропускаем визуализацию BLEU/ROUGE")
        
        # 5. Создание таблицы с примерами
        example_data = []
        for idx, question in enumerate(questions[:5]):  # Ограничиваем до 5 примеров
            try:
                snippets = retrieve_snippets(question, retriever_index, texts, k)
                snippet_texts = [s[:100] + "..." for s, _ in snippets if isinstance(s, str)]
                # Пробуем получить сгенерированный ответ
                generated = generate_answer(question, snippets)
                reference = df['answer'].iloc[idx]
                example_data.append({
                    "Вопрос": question[:100],
                    "Сниппеты": " | ".join(snippet_texts),
                    "Сгенерированный ответ": generated[:100],
                    "Референсный ответ": reference[:100]
                })
            except Exception as e:
                print(f"Ошибка при создании примера {idx}: {e}")
                example_data.append({
                    "Вопрос": question[:100],
                    "Сниппеты": "Ошибка",
                    "Сгенерированный ответ": "Ошибка",
                    "Референсный ответ": df['answer'].iloc[idx][:100]
                })
        
        # Сохранение таблицы
        example_df = pd.DataFrame(example_data)
        example_df.to_csv(os.path.join(output_dir, "rag_examples.csv"), index=False, encoding="utf-8")
        print("Таблица примеров сохранена в rag_examples.csv")
        
    except Exception as e:
        print(f"Ошибка в visualize_results: {e}")

# Выполнение визуализации
try:
    # Если метрики генератора недоступны, передаём None
    bleu_scores = None  # Замените на реальные, если Блок 8 заработает
    rouge_scores = None  # Замените на реальные, если Блок 8 заработает
    visualize_results(df, snippet_index, df['snippet'].tolist(), OUTPUT_DIR, k=5, bleu_scores=bleu_scores, rouge_scores=rouge_scores)
    print("Визуализация завершена, результаты сохранены в", OUTPUT_DIR)
except Exception as e:
    print(f"Ошибка при выполнении визуализации: {e}")

Метрики генератора недоступны, пропускаем визуализацию BLEU/ROUGE
Snippets в generate_answer: [(<class 'tuple'>, ('6. Возмещение за жилое помещение, сроки и другие условия изъятия определяются соглашением с собственником жилого помещения. Принудительное изъятие жилого помещения на основании решения суда возможно только при условии предварительного и равноценного возмещения.; 8. По соглашению с собственником жилого помещения ему может быть предоставлено взамен изымаемого жилого помещения другое жилое помещение с зачетом его стоимости при определении размера возмещения за изымаемое жилое помещение.', 12.02879524230957)), (<class 'tuple'>, ('3. Собственник жилого помещения несет бремя содержания данного помещения и, если данное помещение является квартирой, общего имущества собственников помещений в соответствующем многоквартирном доме, а собственник комнаты в коммунальной квартире несет также бремя содержания общего имущества собственников комнат в такой квартире, если иное не предусмотр

=======================================================================================